<a href="https://colab.research.google.com/github/Keoni808/NFL_Data_Cleaning/blob/main/NFL_Plays_Week1_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PURPOSE:
- To view a larger sample size of plays.
  - Currently working on breaking down a single game but do not have enough data in that game to correctly break down all play descriptions for different play types.

# MOUNTING AND IMPORTS

In [ ]:
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Used to access personal google cloud services
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
# Installs
!pip install ipdb

In [ ]:
# Imports

# Data manipulation
import pandas as pd

# Regular expressions
import re

# Grab data from database
from google.cloud import bigquery

# # Debugging (Not going to use right now)
# import ipdb

In [ ]:
# # Turning on automatic debugger
# %pdb on

# LOADING DATA (BigQuery queries)

In [ ]:
# Client connect to bigquery project
client = bigquery.Client('nfl-data-430702')

## Season 2023 Week 1

In [ ]:
# Grabbing all plays from Super Bowl 2023
week1_2023_plays_query = """
                         SELECT *
                         FROM `nfl-data-430702.NFL_Scores.NFL-Plays-Week1_2023`
                         """

# Running psuedo query, and returns the amount of bytes it will take to run query
dry_run_config = bigquery.QueryJobConfig(dry_run=True)
dry_run_query = client.query(week1_2023_plays_query, job_config=dry_run_config)
print("This query will process {} bytes.".format(dry_run_query.total_bytes_processed))

# Running query (Being mindful of the amount of data being grabbed)
# Will grab a maximum of a Gigabyte
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
safe_config_query = client.query(week1_2023_plays_query, job_config=safe_config)

This query will process 570194 bytes.


In [ ]:
# Putting data attained from query into a dataframe
week1_2023_plays = safe_config_query.to_dataframe()

In [ ]:
week1_2023_plays.head()

,Season,Week,Day,Date,AwayTeam,HomeTeam,Quarter,DriveNumber,TeamWithPossession,IsScoringDrive,PlayNumberInDrive,IsScoringPlay,PlayOutcome,PlayDescription,PlayStart
0,2023,Week 1,MON,09/11,Bills,Jets,1ST QUARTER,1,BUF,0,1,0,Kickoff,G.Zuerlein kicks 65 yards from NYJ 35 to end z...,Kickoff from NYJ 35
1,2023,Week 1,MON,09/11,Bills,Jets,1ST QUARTER,1,BUF,0,2,0,7 Yard Pass,(15:00) (Shotgun) J.Allen pass short right to ...,1st & 10 at BUF 25
2,2023,Week 1,MON,09/11,Bills,Jets,1ST QUARTER,1,BUF,0,3,0,5 Yard Pass,"(14:34) (No Huddle, Shotgun) J.Allen pass shor...",2nd & 3 at BUF 32
3,2023,Week 1,MON,09/11,Bills,Jets,1ST QUARTER,1,BUF,0,4,0,3 Yard Run,(14:01) J.Cook up the middle to BUF 40 for 3 y...,1st & 10 at BUF 37
4,2023,Week 1,MON,09/11,Bills,Jets,1ST QUARTER,1,BUF,0,5,0,2 Yard Run,(13:24) (Shotgun) J.Cook up the middle to BUF ...,2nd & 7 at BUF 40


In [ ]:
# Observation of the amount of data being worked on
week1_2023_plays.shape

(2600, 15)

# CATEGORIZE PLAYS
- The goal here is to parse out the different values for 'PlayOutcome'
  - separate pass / run / kickoff / etc.

## PARSING


In [ ]:
# Maybe try to fuzzywuzzy this in the future?

# All play outcomes from the game
# - From here we can categorize and clean plays accordingly
week1_2023_plays['PlayOutcome'].unique()

array(['Kickoff', '7 Yard Pass', '5 Yard Pass', '3 Yard Run',
       '2 Yard Run', 'Pass Incomplete', 'Punt', '-5 Yard Penalty',
       '5 Yard Run', '1 Yard Pass', '14 Yard Run', '3 Yard Pass',
       '8 Yard Run', '6 Yard Pass', '15 Yard Pass', '-9 Yard Sack',
       '4 Yard Pass', '13 Yard Pass', 'Field Goal', '-2 Yard Sack',
       'Interception', '-5 Yard Run', '18 Yard Pass', '8 Yard Pass',
       '6 Yard Run', '12 Yard Run', '-1 Yard Run', '26 Yard Pass',
       'Touchdown Bills', 'Extra Point Good', '13 Yard Run',
       '-3 Yard Sack', '7 Yard Run', '9 Yard Pass', '4 Yard Run',
       'Fumble', '-10 Yard Penalty', '10 Yard Pass', '26 Yard Run',
       '5 Yard Penalty', '-10 Yard Sack', '22 Yard Pass', '-4 Yard Run',
       '-12 Yard Sack', '83 Yard Run', '1 Yard Run', '2 Yard Pass',
       '10 Yard Run', 'Run for No Gain', '12 Yard Pass', '20 Yard Pass',
       '9 Yard Run', '-2 Yard Pass', 'Sack', '24 Yard Pass',
       '14 Yard Pass', 'Touchdown Jets', '-3 Yard Run', '-2 Yar

In [ ]:
# There are more types of plays that I have not made yet for Week 1.

# Looking at all unique play outcomes and categorizing them.
# - This type of approach does not feel very flexable because a play outcome can
#   arise that has not been seen yet.
# - There may be more in the future when working on a full season, let alone all seasons and future games
df_2023_pass_week1 = week1_2023_plays[week1_2023_plays['PlayOutcome'].str.contains('Pass')]
df_2023_run_week1 = week1_2023_plays[week1_2023_plays['PlayOutcome'].str.contains('Run')]

# df_2023_punt_week1 = week1_2023_plays[week1_2023_plays['PlayOutcome'].str.contains('Punt')]
# df_2023_sack_week1 = week1_2023_plays[week1_2023_plays['PlayOutcome'].str.contains('Sack')]
# df_2023_kickoff_week1 = week1_2023_plays[week1_2023_plays['PlayOutcome'].str.contains('Kickoff')]
# df_2023_fumble_week1 = week1_2023_plays[week1_2023_plays['PlayOutcome'].str.contains('Fumble')]
# df_2023_interception_week1 = week1_2023_plays[week1_2023_plays['PlayOutcome'].str.contains('Interception')]
# df_2023_penalty_week1 = week1_2023_plays[week1_2023_plays['PlayOutcome'].str.contains('Penalty')]
# df_2023_fieldgoal_week1 = week1_2023_plays[week1_2023_plays['PlayOutcome'].str.contains('Field Goal')]
# df_2023_touchdown_week1 = week1_2023_plays[week1_2023_plays['PlayOutcome'].str.contains('Touchdown')]
# df_2023_extrapoint_week1 = week1_2023_plays[week1_2023_plays['PlayOutcome'].str.contains('Extra Point')]

# plays_list = [df_2023_pass_sb,
#               df_2023_run_sb,
#               df_2023_punt_sb,
#               df_2023_sack_sb,
#               df_2023_kickoff_sb,
#               df_2023_fumble_sb,
#               df_2023_interception_sb,
#               df_2023_penalty_sb,
#               df_2023_fieldgoal_sb,
#               df_2023_touchdown_sb,
#               df_2023_extrapoint_sb]

## SANITY CHECK (All Plays Accounted for)
- NOT COMPLETE
  - Still need to grab other play types

## HELPER METHODS (personal use)
- For personal use, does not actually take part in cleaning dataset at all.

In [ ]:
# PURPOSE:
# - Quick look at a section of plays
#   - Ideally the plays that the user wants to break down and clean.
# INPUT PARAMETERS:
# df_all_plays      - DataFrame - The original dataframe where the desired plays to view came from
# df_section_plays  - DataFrame - A section of the original dataframe the user wants to view
# RETURN:
# - Printing to the console:
#   1. index of play
#   2. 'PlayDescription' feature of play
#   3. 'PlayOutcome' feature of play
def print_plays(df_all_plays, df_section_plays):
  for idx, value in df_section_plays['PlayOutcome'].items():
    print("index:" + str(idx))
    play = df_all_plays['PlayDescription'].iloc[idx]
    print(play)
    print(value)
    print()

# PIPELINE

## CLEANING METHODS
- Will contain all methods to clean every type of play
- Uses the raw data in the feature 'PlayOutcome' to parse plays into different categories

In [ ]:
####################################################
# REGULAR EXPRESSIONS USED TO LOCATE SPECIFIC DATA #
####################################################

################
# PLAY DETAILS #
################

time_on_clock_pattern = r'\(\d*:\d+\)'
formation = r'\([A-Za-z]+ ?[A-Za-z]*,? ?[A-Za-z]*\)'
yardage_gained = r'for -?[0-9]+ yards?'

#################
# NAMES OFFENSE #
#################

name_pattern = r'\b[A-Za-z]+\.[A-Za-z]+-?[A-Za-z]*\b' # Grabs all names but will only be used for Passer
receiver_name_pattern = r'\b [A-Za-z]+\.[A-Za-z]+-?[A-Za-z]*\b' # Receivers have a space before their name
rusher_pattern = r'\b[A-Za-z]+\.[A-Za-z]+-?[A-Za-z]* \b' # Runningbacks, like quarterbacks, are the first names in play descriptions

#################
# NAMES DEFENSE #
#################

defense_tackler_1_name_pattern = r'\([A-Za-z]+\.[A-Za-z]+-?[A-Za-z]*' # Will have a "(" in front of the name
defense_tackler_2_name_pattern = r' [A-Za-z]+\.[A-Za-z]+-?[A-Za-z]*\)' # Will have a ")" at the end of the name
defense_pressure_name_pattern = r'\[[A-Za-z]+\.[A-Za-z]+-?[A-Za-z]*\]' # Surrounded by "[]" brackets

########################
# TEAM IDENTIFIED NAME #
########################

team_identified_name = r'-[A-Za-z]+\.[A-Za-z]+-?[A-Za-z]*' # team initials comes before their name (e.g. KC-B.Bob).
                                                           # - This occurs when there is an injury, penalty, fumble recovery.

#######################
# PATTERNS ON FUMBLES #
#######################

qb_fumble = "[A-Za-z]+\.[A-Za-z]+-?[A-Za-z]* to [A-Z]+ [0-9]+ for -?[0-9]+ yards$" # When quarterbacks fumble the ball after snap(?)
run_after_recovery = "^[A-Za-z]+\.[A-Za-z]+-?[A-Za-z]* to [A-Z]+ [0-9]+ for " # yardage after recovery (formatted almost exactly like a regular run play)


### HELPER METHODS (Pass & Run)
- Methods that will help break down unique situations that happen during ordinary plays.
- ADDRESSED SITUATIONS:
  1. Fumbles ( pass / run )
    - Still in the making


##### Pass Fumble Plays

In [ ]:
# THIS IS ONLY FOR PASSING RIGHT NOW.

# Regular expression used to grab QB only fumbles.
# Example - "(14:21) J.Love to CHI 44 for -3 yards"
# NOTE:
# - There are other plays that follow this format.
#   So far I have seen:
#   1. P.Campbell to NYG 33 for -2 yards
#      - What looks like to be an ordinary run play
qb_fumble = "[A-Za-z]+\.[A-Za-z]+-?[A-Za-z]* to [A-Z]+ [0-9]+ for -?[0-9]+ yards$"

# Regular expression used for players who recovered the fumbled ball.
# Example: NYG-P.Campbell
fumble_recoverer = "[A-Z]+-[A-Z]+\.[A-Za-z]+-?[A-Za-z]*"

# PURPOSE:
# - Extract fumble data from fumble plays.
#   - Goal is to strictly grab data that can only appear during fumbled plays,
#     the rest of the data will go down through the pipeline.

def extract_fumble_data(play):

  # Every action of the play is recorded into sentences that can be broken down.
  # - Goal is to strictly grab data that only appears during fumbled plays,
  #   the rest will go through the set play type pipeline.
  play_elements = play.split(". ")
  # Collecting fumble data in the exact order in which it happened.
  extracted_fumble_details = [None] * len(play_elements)
  push_back_to_pipeline = []
  # When traversing through each element, some elements will singal that
  # the next element is a detail exclusively found in fumble plays.
  automatic_fumble_detail_add = False

  for i in play_elements:
    if automatic_fumble_detail_add:
      extracted_fumble_details.pop(play_elements.index(i))
      extracted_fumble_details.insert(play_elements.index(i), i)
      automatic_fumble_detail_add = False
      continue
    else:
      # All plays added to this list, then shaved off if neccessary.
      push_back_to_pipeline.append(i)

    # QB only fumbles
    # (e.g. '(14:21) J.Love to CHI 44 for -3 yards.')
    passer = re.findall(qb_fumble, i)
    if len(passer) == 1:
      # Wanted element (QB only fumble) does not:
      # 1. Follow a sentence stating that the ball has been fumbled.
      #    - In order to check the previous sentence, we must make sure there
      #      is a sentence there to check in the first place.
      if play_elements.index(i) > 0 and play_elements[play_elements.index(i)-1].find('FUMBLES') != -1:
        continue
      else:
        push_back_to_pipeline.pop(push_back_to_pipeline.index(i))
        extracted_fumble_details.pop(play_elements.index(i))
        extracted_fumble_details.insert(play_elements.index(i), i)

    # Fumble and recovery
    # If the person who recovered the ball then goes on to run the ball after,
    # their yardage gained from that run will be automatically added to extracted_fumble_details
    if i.find('FUMBLES') != -1:
      recoverer = re.findall(fumble_recoverer, i)
      if len(recoverer) > 0:
        player_who_recovered_ball = recoverer[0][recoverer[0].find("-") + 1:]
        try:
          if play_elements[play_elements.index(i)+1].find(player_who_recovered_ball) != -1:
            automatic_fumble_detail_add = True
        except IndexError:
          pass
      push_back_to_pipeline.pop(push_back_to_pipeline.index(i))
      extracted_fumble_details.pop(play_elements.index(i))
      extracted_fumble_details.insert(play_elements.index(i), i)

    # Reversed
    # If play has been reversed, the only offensive stats recorded are the
    # sentences that follow the play reversal.
    if i.find('REVERSED') != -1:
      for j in push_back_to_pipeline:
        extracted_fumble_details.pop(play_elements.index(j))
        extracted_fumble_details.insert(play_elements.index(j), j)
      push_back_to_pipeline.clear()

  return extracted_fumble_details, push_back_to_pipeline

In [ ]:
for idx, value in df_2023_pass_week1['PlayOutcome'].items():
  play = week1_2023_plays['PlayDescription'].iloc[idx]
  if play.find('FUMBLES') != -1:
    fumble_details, main_play = extract_fumble_data(play)
    print(fumble_details)
    print(main_play)
    print(". ".join(main_play))
    print(value)
    print()

['(14:21) J.Love to CHI 44 for -3 yards', 'FUMBLES, and recovers at CHI 46', None]
['J.Love pass deep left to L.Musgrave to CHI 4 for 37 yards (T.Stevenson) [D.Walker].']
J.Love pass deep left to L.Musgrave to CHI 4 for 37 yards (T.Stevenson) [D.Walker].
37 Yard Pass

['(14:15) T.Lawrence pass short right to C.Ridley to JAX 47 for 14 yards (R.Thomas, E.Speed)', 'FUMBLES (E.Speed), RECOVERED by IND-E.Speed at IND 49', 'E.Speed ran ob at IND 49 for no gain', 'The Replay Official reviewed the ball was inbounds ruling, and the play was REVERSED', None, 'FUMBLES (E.Speed), ball out of bounds at IND 49', None, None]
['T.Lawrence pass short right to C.Ridley to JAX 47 for 14 yards (R.Thomas, E.Speed)', 'IND-K.Moore was injured during the play', 'IND-D.Flowers was injured during the play.']
T.Lawrence pass short right to C.Ridley to JAX 47 for 14 yards (R.Thomas, E.Speed). IND-K.Moore was injured during the play. IND-D.Flowers was injured during the play.
14 Yard Pass

[None, 'FUMBLES (B.Okere

In [ ]:
for idx, value in df_2023_pass_week1['PlayOutcome'].items():
  play = week1_2023_plays['PlayDescription'].iloc[idx]
  if play.find('FUMBLES') != -1:
    print("index:" + str(idx))
    fumble_play_elements = play.split(". ")
    for i in fumble_play_elements:
      print(i)
    # print(play)
    print(value)
    print()

index:213
(14:21) J.Love to CHI 44 for -3 yards
FUMBLES, and recovers at CHI 46
J.Love pass deep left to L.Musgrave to CHI 4 for 37 yards (T.Stevenson) [D.Walker].
37 Yard Pass

index:423
(14:15) T.Lawrence pass short right to C.Ridley to JAX 47 for 14 yards (R.Thomas, E.Speed)
FUMBLES (E.Speed), RECOVERED by IND-E.Speed at IND 49
E.Speed ran ob at IND 49 for no gain
The Replay Official reviewed the ball was inbounds ruling, and the play was REVERSED
T.Lawrence pass short right to C.Ridley to JAX 47 for 14 yards (R.Thomas, E.Speed)
FUMBLES (E.Speed), ball out of bounds at IND 49
IND-K.Moore was injured during the play
IND-D.Flowers was injured during the play.
14 Yard Pass

index:872
(11:26) (Shotgun) D.Prescott pass short right to T.Pollard to NYG 12 for 7 yards (B.Okereke)
FUMBLES (B.Okereke), recovered by DAL-T.Biadasz at NYG 4.
7 Yard Pass

index:961
(4:45) (Shotgun) D.Jones pass short left to M.Breida to NYG 43 for 5 yards (M.Bell)
FUMBLES (M.Bell), recovered by NYG-P.Campbell at 

##### Run Fumble Plays

In [ ]:
##############################
# MAIN VARIABLES TO FOCUS ON #
##############################

# extracted_fumble_details
# - Extra data that is not practical enough to have it's own feature. Although
#   this data is useful, it is not efficient to have columns for the data
#   that is only available on fumbled plays.
#   - So all this data will be within a list contained within a single feature
#     'extracted_fumble_details'

# push_back_to_pipeline
# - Because this method will serve as a helper method to another, specifically
#   handling fumbled run plays within the set of all run plays, I would like to
#   reduce redundancy by pushing all data that can be cleaned by the main method
#   to that method.
#   - This helper method is simply extracting data that cannot be cleaned within the
#     main method out, packaging and storing it, then sending the rest to the
#     main method.

###################
# DESIGN THOUGHTS #
###################

# - I need the ability to add multiple rows for a single play.
#   When a fumble occurs, there are multiple plays within a single play.
#   1. There is the original play
#      - Currently focusing on run plays
#   2. There is the play after the fumble recovery
#      - Could just be recovered (If so, nothing will follow the fumble action description)
#      - Could have been picked up and rushed for X amount of yards.
# - I have decided that I will collect all data from everyone who
#   touches the ball and carries it.
#   - This means that even if a defender were to recovered a fumbled ball
#     the defender will have rushing yards.

##########
# DESIGN #
##########

# Idea #1

# Variables
# 1. Dataframe_of_plays - Contains all plays
# 2. Dataframe_of_plays_with_added_features - Contains all plays with additional features that serve
#                                             as features that contain specific information gathered
#                                             from a play description.
#                                             EXAMPLE:
#                                             play description - (14:30) J.Doe passes to B.Doe for 13 yards.
#                                             additional features:
#                                             time - (14:30)
#                                             play_type - pass
#                                             Passer - J.Doe
#                                             Receiver - B.Doe
#                                             Yards Gained - 13
#

# Methods

# PIPELINE METHOD
# INPUT - Dataframe_of_plays
# OUTPUT - Dataframe_of_plays_with_added_features

# 1. clean_dataframe_of_plays
#   1.1 Get indexes of a single category of play type within dataframe
#       (e.g. pass, run, touchdown, punt, sack, ... )
#   1.2 Use uniquely designed method for play type to clean within dataframe
#   1.3 Repeat until all plays within dataframe have been cleaned.
#      NOTE:
#      - It is important to fully clean a play type before moving to the next
#        because sometimes cleaning could involve adding a new row to the dataframe,
#        causing a reset to the dataframes indexing.
#        - If we were to separate all play types from the beginning, the indexes
#          could shift around causing an index that might be pointing to a run play
#          to now pointing at a pass play.

# UNIQUE PLAY TYPE CLEANING METHODS
# 1. clean_run_plays
#    - Create a loop that will examine each run play individually
#      - extract data from run play description and fill into added
#        features
#        NOTE:
#        - This play type can add additional rows to overall dataframe
#          - During a fumble, there are potentially 2+ plays here:
#            1. The intended play
#            2. The play after fumble recovery
#               - How I am thinking of handling this right now is by
#                 sending the initial play (1) back to the main stream of
#                 run cleaning (since it follows the same format). Then
#                 see if I can do the same with the ball recovery play. If I am
#                 unable to, then I may have to break it down within the helper method.
#                 - I need to signal that the plays following the initial are
#                   all apart of the same play. So maybe for 'playtype' I can
#                   say something like 'fumble recovery run'.
#
# HOW TO ADD NEW ROW:
# - Once signaled that a new row needs to be made:
#   1. Get index of play that needs new row
#   2. Get data needed for inputs into new row
#   3. Split dataframe to where row needs to be input
#   4. concat before input, input, after input
#   5. reset index.
#     - Issue here is that all plays after this will have a different
#       index. If the loop continues, the indexes will point to the
#       wrong plays.
#       - ~ SOLUTION:
#           - memorize the last index of play that needed the addition of a new row
#             - Relocate all specified type of plays within dataframe and have
#               the cleaning method start where it left off.

In [ ]:
# Version 2
# I think PlayOutcomes are what determine the yardage gained on an intended play.
# Which I do not think is right on their part.

def extract_fumble_data_run(play):
  play_elements = play.split(". ")
  extracted_fumble_details = [None] * len(play_elements)
  push_back_to_pipeline = []
  automatic_fumble_detail_add = False

  for i in play_elements:
    push_back_to_pipeline.append(i)

    # 1. I need to signal that this play has been aborted
    # 2. Should this break down here or in the main breakdown method?
    #    - I am going to try and send it to the main breakdwon method.
    if i.find('Aborted') != -1:
      # push_back_to_pipeline.pop(push_back_to_pipeline.index(i))
      # extracted_fumble_details.pop(play_elements.index(i))
      # extracted_fumble_details.insert(play_elements.index(i), i)
      # Get time
      # Get formation
      # Get quarterback
      continue

    if i.find('FUMBLES') != -1:
      push_back_to_pipeline.pop(push_back_to_pipeline.index(i))
      extracted_fumble_details.pop(play_elements.index(i))
      extracted_fumble_details.insert(play_elements.index(i), i)

    # 1. I need to signal that this play is run after recovery
    # 2. I will break down here
    # 3. I need to add another row if this has been reached.
    #    - Be very mindful about indexing
    after_fumble_action = re.findall(run_after_recovery, i)
    if len(after_fumble_action) > 0:
      # This is where to add another line into the dataframe
      push_back_to_pipeline.pop(push_back_to_pipeline.index(i))
      extracted_fumble_details.pop(play_elements.index(i))
      extracted_fumble_details.insert(play_elements.index(i), i)
      # New row
      continue

    # I think I should handle this in main method.
    if i.find('REVERSED') != -1:
      for j in push_back_to_pipeline:
        extracted_fumble_details.pop(play_elements.index(j))
        extracted_fumble_details.insert(play_elements.index(j), j)
      push_back_to_pipeline.clear()

  return extracted_fumble_details, push_back_to_pipeline

In [ ]:
for idx, value in df_2023_run_week1['PlayOutcome'].items():
  play = week1_2023_plays['PlayDescription'].iloc[idx]
  if play.find('FUMBLES') != -1:
    fumble_details, main_play = extract_fumble_data_run(play)
    fumble_play_elements = play.split(". ")
    for i in fumble_play_elements:
      print(i)
    print("------------")
    print(fumble_details)
    print(main_play)
    main_play = ". ".join(main_play)
    print(main_play)
    # print(value)
    print()
    print()

(9:54) Bre.Hall left end to BUF 22 for -1 yards (G.Rousseau)
FUMBLES (G.Rousseau), ball out of bounds at BUF 25.
------------
[None, 'FUMBLES (G.Rousseau), ball out of bounds at BUF 25.']
['(9:54) Bre.Hall left end to BUF 22 for -1 yards (G.Rousseau)']
(9:54) Bre.Hall left end to BUF 22 for -1 yards (G.Rousseau)


(2:08) S.Clifford FUMBLES (Aborted) at CHI 35, and recovers at CHI 35.
------------
[None]
['(2:08) S.Clifford FUMBLES (Aborted) at CHI 35, and recovers at CHI 35.']
(2:08) S.Clifford FUMBLES (Aborted) at CHI 35, and recovers at CHI 35.


(6:44) (Shotgun) J.Goff Aborted
F.Ragnow FUMBLES at KC 24, recovered by DET-J.Goff at KC 27
J.Goff to KC 27 for no gain (G.Karlaftis).
------------
[None, 'F.Ragnow FUMBLES at KC 24, recovered by DET-J.Goff at KC 27', 'J.Goff to KC 27 for no gain (G.Karlaftis).']
['(6:44) (Shotgun) J.Goff Aborted']
(6:44) (Shotgun) J.Goff Aborted


(8:53) (Shotgun) D.Jones Aborted
J.Schmitz FUMBLES at DAL 18, recovered by NYG-D.Jones at DAL 27.
------------


In [ ]:
for idx, value in df_2023_run_week1['PlayOutcome'].items():
  play = week1_2023_plays['PlayDescription'].iloc[idx]
  print(play)
  print()

(14:01) J.Cook up the middle to BUF 40 for 3 yards (J.Johnson, J.Franklin-Myers).

(13:24) (Shotgun) J.Cook up the middle to BUF 42 for 2 yards (Q.Williams; J.Franklin-Myers).

(9:24) (Shotgun) J.Cook right tackle to BUF 23 for 5 yards (Qu.Williams).

(8:02) (Shotgun) J.Allen scrambles up the middle to BUF 38 for 14 yards (D.Reed).

(6:52) (No Huddle, Shotgun) J.Cook up the middle to BUF 49 for 8 yards (D.Reed).

(10:35) (Shotgun) J.Cook left end to BUF 20 for -5 yards (Q.Williams).

(8:40) (Shotgun) J.Allen scrambles left end pushed ob at NYJ 48 for 6 yards (Q.Jefferson; C.Mosley).

(7:59) J.Cook right end ran ob at NYJ 36 for 12 yards (J.Sherwood).

(7:32) (No Huddle, Shotgun) J.Cook up the middle to NYJ 37 for -1 yards (Qu.Williams, A.Gardner).

(5:34) (Shotgun) D.Harris up the middle to NYJ 5 for 3 yards (Qu.Williams).

(2:36) (Shotgun) J.Cook left tackle pushed ob at NYJ 31 for 13 yards (T.Adams).

(:45) J.Allen up the middle to NYJ 20 for 2 yards (C.Mosley).

(:29) (Shotgun) J.Al

In [ ]:
for idx, value in week1_2023_plays['PlayOutcome'].items():
  play = week1_2023_plays['PlayDescription'].iloc[idx]
  if play.find('REVERSED') != -1:
    print("index:" + str(idx))
    elements = play.split(". ")
    for i in elements:
      print(i)
    print()

index:423
(14:15) T.Lawrence pass short right to C.Ridley to JAX 47 for 14 yards (R.Thomas, E.Speed)
FUMBLES (E.Speed), RECOVERED by IND-E.Speed at IND 49
E.Speed ran ob at IND 49 for no gain
The Replay Official reviewed the ball was inbounds ruling, and the play was REVERSED
T.Lawrence pass short right to C.Ridley to JAX 47 for 14 yards (R.Thomas, E.Speed)
FUMBLES (E.Speed), ball out of bounds at IND 49
IND-K.Moore was injured during the play
IND-D.Flowers was injured during the play.

index:1135
N.Folk kicks 63 yards from TEN 35 to NO 2
R.Shaheed to NO 21 for 19 yards (A.Hooker)
FUMBLES (A.Hooker), ball out of bounds at NO 24
PENALTY on NO-A.Prentice, Offensive Holding, 10 yards, enforced at NO 20
Tennessee challenged the ball was out of bounds ruling, and the play was REVERSED
N.Folk kicks 63 yards from TEN 35 to NO 2
R.Shaheed to NO 21 for 19 yards (A.Hooker)
FUMBLES (A.Hooker), RECOVERED by TEN-A.Hooker at NO 24
Penalty on NO-A.Prentice, Offensive Holding, declined.

index:1566
(2

### RUN PLAYS

In [69]:
# An idea for how to adjust when a row is added to df
# index_start - Can be changed if some rows have already been cleaned

# def clean_run_plays(df_plays, index_start=0)

def clean_run_plays(df_plays):

  # All run plays within df_plays
  df_run_plays = df_plays[df_plays['PlayOutcome'].str.contains('Run')]

  # Iterating through all run plays
  for idx, value in df_run_plays['PlayOutcome'].items():
    play = df_plays['PlayDescription'].iloc[idx]

    ################
    # Play details #
    ################

    # Play Type
    # Should be different for aborted or fumble plays?
    df_plays.loc[idx, 'PlayType'] = 'Run'

    # TimeOnTheClock
    TimeOnTheClock = re.findall(time_on_clock_pattern, play)
    df_plays.loc[idx, 'TimeOnTheClock'] = TimeOnTheClock[0][1:-1]

    ############
    # REVERSES #
    ############

    # We do not need to record any information before the reversed statement
    # within the play description.
    if play.find('REVERSED') != -1:
      play_elements = play.split(". ")
      for i in play_elements:
        if i.find("REVERSED") != -1:
          df_plays.at[idx, 'ReverseDetails'] = play_elements[:play_elements.index(i) + 1]
          play = ". ".join(play_elements[play_elements.index(i) + 1:])
          break

    ############################
    # REPORTING IN AS ELIGIBLE #
    ############################

    # I do not think this contains any useful data so I am going to exclude it.
    if play.find('reported in as eligible') != -1:
      play_elements = play.split(". ")
      for i in play_elements:
        if i.find('reported in as eligible') != -1:
          play = ". ".join(play_elements[play_elements.index(i) + 1:])
          break

    # Handle Fumbles
    if play.find('FUMBLES') != -1:
      # Method for fumble
      fumble_details, play = extract_fumble_data_run(play)
      df_plays.at[idx, 'FumbleDetails'] = fumble_details
      play = ". ".join(play)
      # continue

    yardage = re.findall(yardage_gained, play)
    if len(yardage) > 0:
      df_plays.loc[idx, 'Yardage'] = int(yardage[0].split()[1])
    else:
      df_plays.loc[idx, 'Yardage'] = 0

    ##########################
    # MISTAKES AND TURNOVERS #
    ##########################

    if play.find('PENALTY') != -1:
      # Method for penalty
      continue

    #############
    #  OFFENSE  #
    #############

    # Formation
    Formation = re.findall(formation, play)
    if len(Formation) > 0:
      df_plays.loc[idx, 'Formation'] = Formation[0][1:-1]
    # Rusher
    rusher_names = re.findall(rusher_pattern, play) # May grab name(s) bc regular expression. (Only want rusher)
    rusher_name = rusher_names[0][:-1]
    df_plays.loc[idx, 'Rusher'] = rusher_name
    # Direction
    rushing_directions = ['guard', 'middle', 'tackle', 'end', 'kneels']
    for i in rushing_directions:
      if play.find(i) != -1:
        start = play.find(rusher_name) + len(rusher_name) + 1
        end = play.find(i) + len(i)
        df_plays.loc[idx, 'Direction'] = play[start:end]

    #############
    #  DEFENSE  #
    #############

    tackler_1 = re.findall(defense_tackler_1_name_pattern, play) # tackler #1 (Could be solo or the one who initiated the hit)
    if len(tackler_1) > 0:
      df_plays.loc[idx, 'TackleBy1'] = tackler_1[0][1:]
    tackler_2 = re.findall(defense_tackler_2_name_pattern, play) # tackler #2 (equally contributed or assisted with tackle)
    if len(tackler_2) > 0:
      df_plays.loc[idx, 'TackleBy2'] = tackler_2[0][1:-1]

## PIPELINE MAIN METHOD
- Method that wraps all unique play type cleaning methods into one.

In [70]:
# Pipeline method I will use that will simple accept a dataframe of plays
# and return a dataframe with those plays cleaned.

def clean_dataframe_of_plays(df_all_plays):

  ###########################
  # NEW COLUMN DESCRIPTIONS #
  ###########################

  # PlayType           - The type of play (e.g. pass/run)
  # TimeOnTheClock     - The time that was on the clock when the play started
  # Formation          - Play formation
  # Passer             - Player that threw the ball (mostly the quarterback)
  # Rusher             - Player that ran the ball (mostly the runningback)
  # Receiver           - Player on the same team as the passer that caught the ball
  # PassType           - Whether the pass was a deep or short pass?
  # Direction          - Where the ball is going during the play
  # Yardage            - Yards gained during the play
  # TackleBy1          - Main tackler on the play (could be solo or could be with someone else)
  # TackleBy2          - Assisted tackler1
  # PressureBy         - Defender that applied pressure to the passer
  # ForcedFumbleBy     - Defender that forced a fumble
  # AfterFumble        - A list that has what happened after the fumble
  #                      - [recovered by, yards gained, tackled by]
  # InjuredPlayers     - Players that were injured during the play
  # PenaltyDescription - If there is a penalty, gives a description of it
  #                      - [who caused the penalty, what was the penalty, yards lost if penalty accepted]
  # Yardage            - Total yardage gained on intended play (yardage gained from penalties and fumble recoveries do not count)

  new_columns = ["PlayType", "TimeOnTheClock", "Formation", "Passer", "Rusher", "Receiver", "PassType", "Direction", "Yardage",
                "TackleBy1", "TackleBy2", "PressureBy", "ForcedFumbleBy",
                "FumbleDetails", "ReverseDetails",
                "InjuredPlayers", "PenaltyDescription"]

  string_columns = ["PlayType", "TimeOnTheClock", "Formation", "Passer", "Rusher", "Receiver", "PassType", "Direction",
                    "TackleBy1", "TackleBy2", "PressureBy", "ForcedFumbleBy",
                    "FumbleDetails", "ReverseDetails",
                    "InjuredPlayers", "PenaltyDescription"]

  int_columns = ["Yardage"]

  ########################################
  # RETURN DATAFRAME WITH ADDED FEATURES #
  ########################################

  df_all_plays_cleaned = df_all_plays.copy()
  df_all_plays_cleaned = df_all_plays_cleaned.reindex(columns=df_all_plays_cleaned.columns.tolist() + new_columns)
  df_all_plays_cleaned[string_columns] = df_all_plays_cleaned[string_columns].astype(str)
  df_all_plays_cleaned[int_columns] = df_all_plays_cleaned[int_columns].astype(float)

  ########################################
  # GETTING PLAY CATEGORIES AND CLEANING #
  ########################################

  clean_run_plays(df_all_plays_cleaned)

  return df_all_plays_cleaned

In [71]:
df = clean_dataframe_of_plays(week1_2023_plays)

In [55]:
bills_jets_run_plays = df.loc[(df['PlayType'] == 'Run') & (df['AwayTeam'] == 'Bills')]

In [56]:
bills_jets_run_plays['Rusher'].unique()

array(['J.Cook', 'J.Allen', 'D.Harris', 'D.Harty', 'L.Murray', 'Bre.Hall',
       'D.Cook', 'Z.Wilson', 'Mi.Carter'], dtype=object)

In [57]:
bills_jets_run_plays['Yardage'].loc[bills_jets_run_plays['Rusher'] == 'Bre.Hall'].sum()

130.0

In [58]:
df['Yardage'].unique()

array([nan,  3.,  2.,  5., 14.,  8., -5.,  6., 12., -1., 13.,  7.,  4.,
       26., -4., 83.,  1., 10.,  0.,  9., -3., -2., 11., -7., 22., 17.,
       19., 16., 18., 25., 21., 55., -6., 15., 24., 29.])